In [1]:
!pip install hyperopt

In [17]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix_three/dw_matrix_road_sign"

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

In [0]:
X_train, y_train = train['features'],train['labels']
X_test, y_test = test['features'],test['labels']

In [0]:
if y_test.ndim == 1 : y_test = to_categorical(y_test)
if y_train.ndim == 1 : y_train = to_categorical(y_train)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size =  params_fit.get('batch_size',128),
      epochs =  params_fit.get('epochs',5),
      verbose =  params_fit.get('verbose',1),
      validation_data =  params_fit.get('validation_data',(X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model, logdir


In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
predict(model_trained, X_test, y_test)

0.854421768707483

In [0]:
df_signnames = pd.read_csv('data/dw_signnames.csv')
labels_dict = df_signnames.to_dict()['b']

In [0]:
def get_cnn_v5(imput_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),
                     
                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(num_classes, activation='softmax')
  ])

In [13]:
model_cnn_5 = get_cnn_v5(input_shape, num_classes)
model_trained, logdir = train_model(model_cnn_5, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.6288 - accuracy: 0.2887 - val_loss: 1.0072 - val_accuracy: 0.6800
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.9376 - accuracy: 0.7019 - val_loss: 0.3504 - val_accuracy: 0.8861
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.5117 - accuracy: 0.8356 - val_loss: 0.1553 - val_accuracy: 0.9522
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3529 - accuracy: 0.8889 - val_loss: 0.0848 - val_accuracy: 0.9738
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2604 - accuracy: 0.9187 - val_loss: 0.0697 - val_accuracy: 0.9776


0.9532879818594104

In [0]:
model_trained.evaluate(X_test,y_test)[1]

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size =  int(params.get('batch_size',128)),
      epochs =  7,
      verbose =  0
      )
  
  accurancy = model.evaluate(X_test, y_test)[0]
  print(params, 'accurancy = {}'.format(accurancy))

  return {'loss': -accurancy, 'status': STATUS_OK, 'model': model}


In [26]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.quniform('dropout_cnn_block_one', 0, 1, 0.1),
    'dropout_cnn_block_two': hp.quniform('dropout_cnn_block_two', 0, 1, 0.1),
    'dropout_cnn_block_three': hp.quniform('dropout_cnn_block_three', 0, 1, 0.1),
    'dropout_dense_block_one': hp.quniform('dropout_dense_block_one', 0, 1, 0.1),
    'dropout_dense_block_two': hp.quniform('dropout_dense_block_two', 0, 1, 0.1)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.6941 - accuracy: 0.7188

 11/138 [=>............................]
 - ETA: 0s - loss: 0.1467 - accuracy: 0.9489

 23/138 [====>.........................]
 - ETA: 0s - loss: 0.3834 - accuracy: 0.9253

 36/138 [======>.......................]
 - ETA: 0s - loss: 0.2477 - accuracy: 0.9523

 47/138 [=========>....................]
 - ETA: 0s - loss: 0.2722 - accuracy: 0.9415

 57/138 [===========>..................]
 - ETA: 0s - loss: 0.2298 - accuracy: 0.9496


In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),
                     
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),

                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_one']),
                     
                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_two']),

                    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size =  int(params.get('batch_size',128)),
      epochs = 5,
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accurancy = score[1]
  print(params, 'accurancy = {}'.format(accurancy))

  return {'loss': -accurancy, 'status': STATUS_OK, 'model': model}


In [40]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.36603049033869745, 'dropout_cnn_block_three': 0.3025923164426087, 'dropout_cnn_block_two': 0.3773558172098523, 'dropout_dense_block_one': 0.6104789728250022, 'dropout_dense_block_two': 0.4817361531653769}
accurancy = 0.9705215692520142
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3998790890173108, 'dropout_cnn_block_three': 0.42930297871619283, 'dropout_cnn_block_two': 0.30524957895175614, 'dropout_dense_block_one': 0.6546456797837252, 'dropout_dense_block_two': 0.4910373884883248}
accurancy = 0.9544217586517334
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3954525699494767, 'dropout_cnn_block_three': 0.3252795550312128, 'dropout_cnn_block_two': 0.3325589821856178, 'dropout_dense_block_one': 0.5707248691089148, 'dropout_dense_block_two': 0.39596492583029097}
accurancy = 0.9442176818847656
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.39256469779228215, 'dropout_cnn_block_three': 0.4412176095719683, 'dropout_cnn_block_two': 0.48